# Lesson 9: Distributed Multi-Agent Systems

This interactive notebook teaches you how to build distributed agent architectures with cross-platform communication:

- ✅ Agents-as-Tools pattern for hierarchical orchestration
- ✅ A2A Server for exposing agents over HTTP
- ✅ A2A Client for remote agent communication
- ✅ Combined patterns for distributed systems
- ✅ Best practices for agent discovery and error handling

**Estimated time:** 6-7 hours

**What you'll build:** Hierarchical orchestrators, HTTP-exposed agents, and distributed agent networks!

## Setup

Import necessary modules and configure the environment:

In [ ]:
from lesson_utils import load_environment, create_working_model, check_api_keys
from strands import Agent, tool

# Load environment and check API keys
load_environment()
check_api_keys()

print("🎯 Lesson 9: Distributed Multi-Agent Systems")
print("=" * 50)

## Part 1: Agents as Tools Pattern - Hierarchical Orchestration

The **Agents-as-Tools pattern** enables hierarchical orchestration where:
- **Primary orchestrator** handles user interaction and routing
- **Specialized tool agents** perform domain-specific tasks
- **Clear delegation chain** with focused responsibilities
- **Modular architecture** allows independent agent modifications

### When to Use Agents-as-Tools:
- Complex queries requiring multiple domains of expertise
- Customer service with specialized departments
- Research systems with different specialist roles
- Any scenario where coordination is needed between specialists

### Architecture:
```
User → Orchestrator Agent → Specialist Agent Tools
                           (research, analysis, creative)
```

**Benefits:**
- Separation of concerns (focused responsibility per agent)
- Hierarchical delegation (clear chain of command)
- Modular architecture (specialists modified independently)
- Improved performance (tailored prompts and tools)

**Reference:** [Strands Agents as Tools Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agents-as-tools/)

In [ ]:
# Multi-agent workflows need higher max_tokens (1500 vs default 500)
model = create_working_model("multiagent")

if model:
    print("\n🏗️  Creating specialized agent tools...\n")

    # =================================================================
    # Step 1: Define Specialized Agent Tools
    # =================================================================
    # Each specialist is wrapped as a @tool function that creates and
    # invokes a specialized Agent with focused system prompts.

    @tool
    def research_assistant(query: str) -> str:
        """
        Process research-related queries requiring factual information.

        Use this tool when the user asks for:
        - Historical facts and dates
        - Scientific explanations
        - Statistical data
        - General knowledge questions

        Args:
            query: A research question requiring factual information

        Returns:
            A detailed research answer with explanations
        """
        try:
            research_agent = Agent(
                model=model,
                name="research_specialist",
                system_prompt="""You are a specialized research assistant.
                Focus only on providing factual, well-sourced information.
                Be concise but thorough. Explain your reasoning.
                If you're unsure, say so - don't make up information."""
            )
            response = research_agent(query)
            return str(response)
        except Exception as e:
            return f"Error in research assistant: {str(e)}"

    @tool
    def data_analyst(query: str) -> str:
        """
        Perform data analysis, calculations, and quantitative reasoning.

        Use this tool when the user asks for:
        - Mathematical calculations
        - Data interpretation
        - Statistical analysis
        - Quantitative comparisons

        Args:
            query: A data analysis question requiring quantitative reasoning

        Returns:
            Analysis results with calculations and interpretations
        """
        try:
            analyst_agent = Agent(
                model=model,
                name="data_analyst_specialist",
                system_prompt="""You are a specialized data analyst.
                Focus on quantitative analysis, calculations, and data interpretation.
                Show your work step-by-step. Explain your methodology.
                Use clear numerical formatting."""
            )
            response = analyst_agent(query)
            return str(response)
        except Exception as e:
            return f"Error in data analyst: {str(e)}"

    @tool
    def creative_writer(query: str) -> str:
        """
        Generate creative content like stories, poems, or marketing copy.

        Use this tool when the user asks for:
        - Creative writing
        - Storytelling
        - Marketing copy
        - Poetic or artistic content

        Args:
            query: A creative writing request

        Returns:
            Original creative content
        """
        try:
            writer_agent = Agent(
                model=model,
                name="creative_writer_specialist",
                system_prompt="""You are a specialized creative writer.
                Focus on engaging, original content with vivid language.
                Be imaginative and expressive. Consider tone and audience.
                Create compelling narratives or copy."""
            )
            response = writer_agent(query)
            return str(response)
        except Exception as e:
            return f"Error in creative writer: {str(e)}"

    print("✓ Created 3 specialist agent tools: research, analysis, creative\n")

    # =================================================================
    # Step 2: Create Orchestrator Agent
    # =================================================================
    # The orchestrator has clear routing logic in its system prompt and
    # access to all specialist agents as tools.

    print("🎯 Creating orchestrator agent with routing logic...\n")

    orchestrator = Agent(
        model=model,
        name="orchestrator",
        system_prompt="""You are an intelligent orchestrator that routes queries
        to specialized agents:

        - For factual questions, history, science → Use research_assistant
        - For calculations, data analysis, numbers → Use data_analyst
        - For stories, creative content, marketing → Use creative_writer
        - For simple greetings or clarifications → Answer directly

        Always select the most appropriate specialist based on the query type.
        You can use multiple specialists if needed for complex queries.""",
        tools=[research_assistant, data_analyst, creative_writer]
    )

    print("✓ Orchestrator created with 3 specialist tools\n")

    # =================================================================
    # Step 3: Test Hierarchical Orchestration
    # =================================================================
    print("📋 Testing queries that require different specialists...\n")

    test_queries = [
        ("Research", "What caused the fall of the Roman Empire?"),
        ("Analysis", "If I invest $1000 at 5% annual interest for 3 years, how much will I have?"),
        ("Creative", "Write a short haiku about artificial intelligence"),
    ]

    for category, query in test_queries:
        print(f"[{category}] Query: {query}")
        print("-" * 70)

        try:
            response = orchestrator(query)
            print(f"Response: {response}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")

    print("\n💡 Key Concepts:")
    print("   • Each specialist agent has a focused system prompt and responsibility")
    print("   • The @tool decorator wraps agents as callable functions")
    print("   • The orchestrator intelligently routes to the right specialist")
    print("   • Specialists can be modified independently without affecting others")
    print("   • This pattern scales well with many specialized agents")

else:
    print("⚠️ No API key available")

## Part 2: A2A Server - Exposing Agents over HTTP

The **A2A (Agent-to-Agent) protocol** enables exposing agents over HTTP for cross-platform communication.

### What is A2A?
A2A is an open standard defining how AI agents discover, communicate, and collaborate across platforms.

### Use Cases:
- **Multi-Agent Workflows** - Chain specialized agents together
- **Agent Marketplaces** - Discover and use agents from different providers
- **Cross-Platform Integration** - Connect Strands agents with other A2A systems
- **Distributed AI** - Build scalable, distributed agent architectures

### A2A Server Features:
- Agent card discovery (metadata about capabilities)
- SendMessageRequest (non-streaming)
- SendStreamingMessageRequest (streaming responses)
- Configurable endpoints and authentication

**Reference:** [Strands A2A Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/)

In [ ]:
print("\n📦 A2A Server Installation:")
print("   To use A2A functionality, install with the a2a extra:")
print("   $ pip install 'strands-agents[a2a]'")
print("   or")
print("   $ uv pip install 'strands-agents[a2a]'")

print("\n🔍 Checking A2A availability...")

try:
    from strands.multiagent.a2a import A2AServer
    print("✓ A2A dependencies are installed")
    
    model = create_working_model("multiagent")

    if model:
        print("\n🏗️  Creating an agent to expose as A2A server...")

        calculator_agent = Agent(
            model=model,
            name="Calculator Agent",
            description="A calculator agent that can perform arithmetic operations.",
            system_prompt="""You are a helpful calculator assistant.
            Perform arithmetic calculations accurately and explain your work.
            If asked non-math questions, politely redirect to mathematical queries."""
        )

        print("✓ Created calculator agent")

        print("\n🌐 Creating A2A server...")
        print("   Server will expose the agent at: http://127.0.0.1:9000")

        a2a_server = A2AServer(
            agent=calculator_agent,
            host="127.0.0.1",
            port=9000
        )

        print("✓ A2A server created")

        print("\n📚 How to start the server:")
        print("   In a separate terminal or background process:")
        print("   ")
        print("   from strands import Agent")
        print("   from strands.multiagent.a2a import A2AServer")
        print("   ")
        print("   agent = Agent(...)")
        print("   server = A2AServer(agent=agent)")
        print("   server.serve()  # Starts the HTTP server")
        print("   ")
        print("   The server supports:")
        print("   • Agent card discovery at /.well-known/agent-card.json")
        print("   • SendMessageRequest (non-streaming)")
        print("   • SendStreamingMessageRequest (streaming)")

        print("\n💡 Advanced Server Features:")
        print("   • Custom middleware via to_fastapi_app() or to_starlette_app()")
        print("   • Path-based mounting for load balancers (http_url parameter)")
        print("   • Custom task storage with task_store parameter")
        print("   • Custom queue management with queue_manager parameter")

        print("\n⚠️  Note: This example doesn't start the server to avoid blocking.")
        print("   See experiments section for a complete server implementation.")
    else:
        print("⚠️ No API key available")
        
except ImportError:
    print("✗ A2A dependencies not found")
    print("\n⚠️  To run A2A examples:")
    print("   1. Install A2A dependencies: uv pip install 'strands-agents[a2a]'")
    print("   2. Re-run this lesson")
    print("\n   Skipping A2A server example for now...")

## Part 3: A2A Client - Remote Agent Communication

The **A2A Client** enables connecting to and invoking remote agents via the A2A protocol.

### Client Types:

1. **Synchronous Client** - Single request-response
2. **Streaming Client** - Real-time streamed responses
3. **Tool Wrapper** - Use remote agents as local tools

### Key Components:
- **Agent Card** - Metadata describing agent capabilities (auto-discovered)
- **Client Factory** - Creates appropriate client based on agent card
- **Message Protocol** - Standardized message format across platforms

**Reference:** [Strands A2A Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/)

In [ ]:
print("\n🔍 Checking A2A client availability...")

try:
    from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
    from a2a.types import Message, Part, Role, TextPart
    print("✓ A2A client dependencies are installed")
    
    print("\n📚 A2A Client Patterns:")
    print("\n1. Synchronous Client (request-response):")
    print("   async def send_sync_message(message, base_url='http://127.0.0.1:9000'):")
    print("       async with httpx.AsyncClient() as client:")
    print("           resolver = A2ACardResolver(httpx_client=client, base_url=base_url)")
    print("           agent_card = await resolver.get_agent_card()")
    print("           ")
    print("           config = ClientConfig(httpx_client=client, streaming=False)")
    print("           factory = ClientFactory(config)")
    print("           a2a_client = factory.create(agent_card)")
    print("           ")
    print("           msg = Message(kind='message', role=Role.user, parts=[...])")
    print("           async for event in a2a_client.send_message(msg):")
    print("               return event  # Single response")

    print("\n2. Streaming Client (real-time responses):")
    print("   # Same as above but with streaming=True")
    print("   config = ClientConfig(httpx_client=client, streaming=True)")
    print("   # Iterate through multiple events as they stream")

    print("\n3. A2A Client Tool Provider (easiest):")
    print("   from strands_tools.a2a_client import A2AClientToolProvider")
    print("   ")
    print("   provider = A2AClientToolProvider(")
    print("       known_agent_urls=['http://127.0.0.1:9000']")
    print("   )")
    print("   agent = Agent(tools=provider.tools)")
    print("   # Agent can now discover and interact with A2A servers")

    print("\n💡 Key A2A Client Concepts:")
    print("   • Agent Card: Metadata describing agent capabilities (auto-discovered)")
    print("   • Client Factory: Creates appropriate client based on agent card")
    print("   • Message Protocol: Standardized message format across platforms")
    print("   • Streaming vs Sync: Choose based on response time expectations")

    print("\n⚠️  Note: A2A clients require a running A2A server to connect to.")
    print("   See experiments section for complete client/server implementations.")
    
except ImportError:
    print("✗ A2A client dependencies not found")
    print("\n⚠️  To run A2A client examples:")
    print("   Install: uv pip install 'strands-agents[a2a]'")
    print("\n   Skipping A2A client example for now...")

## Part 4: Combined Pattern - Distributed Architecture

Combining **Agents-as-Tools** with the **A2A protocol** creates powerful distributed architectures.

### Architecture:
```
User → Local Orchestrator → Local Specialists (Agents-as-Tools)
                          \→ Remote Specialists (A2A Clients)
```

### Benefits:
- **Flexible deployment** - Local vs remote based on requirements
- **Scalability** - Distribute compute-intensive agents
- **Integration** - Connect agents across different platforms
- **Reusability** - Share specialist agents across systems

### When to Use:
- ✓ Some agents need expensive compute (GPUs, large models)
- ✓ Some agents have proprietary data/access
- ✓ Need to integrate agents from different providers
- ✓ Want to share specialized agents across multiple systems
- ✓ Need geographic distribution for latency/compliance

### Implementation Approaches:

1. **Class-based A2A Tool** (Recommended)
   - Discover agent card once during initialization
   - Wrap as @tool for orchestrator
   - Reduces repeated discovery overhead

2. **A2AClientToolProvider** (Quick Start)
   - Automatic agent discovery
   - Natural language interface
   - Good for dynamic agent marketplaces

3. **Direct A2A Client** (Full Control)
   - Manual message construction
   - Custom error handling
   - Advanced streaming scenarios

In [ ]:
print("\n🏗️  Real-World Distributed Architecture:")
print("\n   Scenario: Research & Analysis System")
print("   • Local Orchestrator (this system)")
print("   • Local Agent: Quick data analyst")
print("   • Remote Agent: Heavy computation research engine (A2A)")
print("   • Remote Agent: Specialized domain expert (A2A)")

print("\n📐 Architecture Pattern:")
print("   ")
print("   ┌─────────────────────────────────────────┐")
print("   │         User Application                │")
print("   └──────────────┬──────────────────────────┘")
print("                  │")
print("   ┌──────────────▼──────────────────────────┐")
print("   │    Local Orchestrator Agent             │")
print("   │    (Agents-as-Tools pattern)            │")
print("   └──┬──────────────────┬───────────────────┘")
print("      │                  │")
print("   ┌──▼─────────┐   ┌───▼──────────────────┐")
print("   │ Local      │   │ Remote Agents (A2A)  │")
print("   │ Specialist │   │ • Research Engine    │")
print("   │ Tools      │   │ • Domain Expert      │")
print("   └────────────┘   │ • Compute Cluster    │")
print("                    └──────────────────────┘")

print("\n📊 Design Considerations:")
print("   • Latency: Remote agents add network overhead")
print("   • Reliability: Handle network failures gracefully")
print("   • Security: Authenticate remote agent access")
print("   • Cost: Consider compute/API costs per agent")
print("   • Monitoring: Track performance of distributed calls")

## Experiments

Now it's your turn! Try these experiments:

### Exercises:
1. **Add More Specialists** - Create translator, code reviewer, or documentation writer agents
2. **Multi-Tier Orchestration** - Build an orchestrator of orchestrators (nested delegation)
3. **Error Handling** - Implement retries and fallback logic for specialist calls
4. **Complete A2A System** - Build a working A2A server and client pair
5. **Class-Based A2A Tool** - Create a tool that caches agent cards for efficiency
6. **Fallback Logic** - Handle scenarios when remote agents are unavailable
7. **Monitoring** - Add logging and metrics for distributed agent calls
8. **Load Balancer** - Distribute requests across multiple remote agent instances
9. **Authentication** - Implement auth for A2A server endpoints
10. **Real-World System** - Build a complete distributed research pipeline

Use the cell below for your experiments:

In [ ]:
# Your experiments here!


## ✅ Success Criteria

You've completed Lesson 9 if:

- ✅ Understand Agents-as-Tools pattern for hierarchical orchestration
- ✅ Can wrap specialized agents as tools with @tool decorator
- ✅ Built orchestrator that intelligently routes to specialists
- ✅ Understand A2A Server for exposing agents over HTTP
- ✅ Understand A2A Client for connecting to remote agents
- ✅ Can design distributed architectures combining local and remote agents
- ✅ Know when to use each distributed agent pattern

## 💡 Key Concepts Learned

- **Agents-as-Tools** - Wrapping specialized agents as callable tools for orchestrators
- **Hierarchical Delegation** - Clear chain of command from orchestrator to specialists
- **A2A Protocol** - Open standard for cross-platform agent communication
- **A2A Server** - Exposing agents via HTTP with agent card discovery
- **A2A Client** - Connecting to remote agents (sync vs streaming)
- **Distributed Architecture** - Combining local and remote agents effectively
- **Pattern Selection** - Choosing between local tools vs remote A2A agents

## Next Steps

- **Lesson 10**: Production - Safety, security, observability, and agent evaluation

Ready to make your agents production-ready? Open `lesson_10_production.ipynb`!